In [15]:
#KEY GENERATION::

import random
from hashlib import sha256

p = int(input("Enter a prime number (23,19,17 etc): "))
q = int(input("Enter different prime number: ")) 

#for example we can take 17 and 61 as prime numbers
    #p = 17
    #q= 61

    
# gives the extended greatest common divisor of the integers
# which computes, besides the greatest common divisor of integers a and b, 
# the coefficients of Bezout’s identity
def extended_gcd(aa, bb):
    lastremainder, remainder = abs(aa), abs(bb)
    x, lastx, y, lasty = 0, 1, 1, 0
    while remainder:
        lastremainder, (quotient, remainder) = remainder, divmod(lastremainder, remainder)
        x, lastx = lastx - quotient*x, x
        y, lasty = lasty - quotient*y, y
    return lastremainder, lastx * (-1 if aa < 0 else 1), lasty * (-1 if bb < 0 else 1)

#Euclid's extended algorithm for finding the multiplicative inverse of two numbers 
# Its is use to gererate generate the private key in rsa algorithm
def modinv(a, m):
	g, x, y = extended_gcd(a, m)
	if g != 1:
		raise Exception('Modular inverse does not exist')
	return x % m    


#Check if the num is prime or not
def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2):
        if num % n == 0:
            return False
    return True


# Euclid's Algorithm to verify that two number is coprime wrt each other 

def coprime(a, b):
    while b != 0:
        a, b = b, a % b
    return a
    
#generate key pair using RSA algorithm
def generate_keypair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')

    #we need to generate the value of n using the prime numbers that we have choosen
    n = p * q

    #then calculate the value of Phi that is the totient of n
    phi = (p-1) * (q-1)

    #NOw We need to Choose an integer e such that e and phi(n) are coprime
    e = random.randrange(1, phi)

    #we Use Euclid's Algorithm to verify that e and phi(n) are comprime 
    g = coprime(e, phi)
  
    while g != 1:
        e = random.randrange(1, phi)
        g = coprime(e, phi)

    #Use Extended Euclid's Algorithm to generate the private key
    d = modinv(e, phi)

    #Return public and private keypair
    #Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))
    
print("Process of Generating public/private keypairs starts ")
public, private = generate_keypair(p, q)
    
print("public key :: ", public)
print("private key :: ", private)

Enter a prime number (23,19,17 etc): 17
Enter different prime number: 61
Process of Generating public/private keypairs starts 
public key ::  (167, 1037)
private key ::  (23, 1037)


In [16]:
# ENCRYPTION

# outputs a 256 bits long hash value.
def hashFunction(message):
    hashed = sha256(message.encode("UTF-8")).hexdigest()
    return hashed


def encrypt(privatek, plaintext):
    #privatek key is Unpacked into it's components
    key, n = privatek

    #based on the character using a^b mod m, we convert each letter in the plaintext to numbers  
    numberRepr = [ord(char) for char in plaintext]
    print("Number representation before encryption: ", numberRepr)
    cipher = [pow(ord(char),key,n) for char in plaintext]
    
    #array of bytes is Returned
    return cipher

#message is encrypted using private key of sender
message = input("Enter a message to encrypt with private key: ")
    
print("")

hashed = hashFunction(message)
    
print("Encrypting message with private key ", private ," . . .")
    
encrypted_msg = encrypt(private, hashed)   
    
print("Encrypted hashed message:: ")
print(''.join(map(lambda x: str(x), encrypted_msg)))
#print(encrypted_msg)
    
print("")

Enter a message to encrypt with private key: this is message

Encrypting message with private key  (23, 1037)  . . .
Number representation before encryption:  [53, 54, 57, 99, 56, 54, 56, 101, 50, 48, 100, 57, 98, 101, 56, 98, 97, 50, 56, 102, 98, 53, 53, 56, 100, 102, 56, 56, 57, 51, 98, 54, 48, 97, 101, 102, 98, 99, 50, 99, 48, 57, 98, 56, 54, 49, 53, 56, 100, 57, 52, 50, 53, 53, 52, 57, 54, 50, 98, 57, 49, 101, 50, 101]
Encrypted hashed message:: 
281776388100946977646988356038011038863388346963343256046935763328128146911035746946938896963377638043288335763310095601009380388633469776960281469110388613560281281613388776560633388960883560883



In [17]:
#DECRYPTION

def decrypt(publick, ciphertext):
    #publick key is Unpacked into it's components
    key, n = publick
       
    #based on the ciphertext and key using a^b mod m, Generate the plaintext 
    numberRepr = [pow(char, key, n) for char in ciphertext]
    plain = [chr(pow(char, key, n)) for char in ciphertext]

    print("Decrypted number representation is: ", numberRepr)
    
    #array of bytes is Returned as a string
    return ''.join(plain)

print("Decrypting message with public key ", public ," . . .")

decrypted_msg = decrypt(public, encrypted_msg)
print("Decrypted message::")  
print(decrypted_msg)

Decrypting message with public key  (167, 1037)  . . .
Decrypted number representation is:  [53, 54, 57, 99, 56, 54, 56, 101, 50, 48, 100, 57, 98, 101, 56, 98, 97, 50, 56, 102, 98, 53, 53, 56, 100, 102, 56, 56, 57, 51, 98, 54, 48, 97, 101, 102, 98, 99, 50, 99, 48, 57, 98, 56, 54, 49, 53, 56, 100, 57, 52, 50, 53, 53, 52, 57, 54, 50, 98, 57, 49, 101, 50, 101]
Decrypted message::
569c868e20d9be8ba28fb558df8893b60aefbc2c09b86158d942554962b91e2e


In [18]:
#VERIFICATION

#verification process of received hashed value and message
def verify(receivedHashed, message):
    ourHashed = hashFunction(message)
    if receivedHashed == ourHashed:
        print("Verification status:: successful: ", )
        print(receivedHashed, " = ", ourHashed)
    else:
        print("Verification status:: failed")
        print(receivedHashed, " != ", ourHashed)
        
print("Verification process::")
verify(decrypted_msg, message)

Verification process::
Verification status:: successful: 
569c868e20d9be8ba28fb558df8893b60aefbc2c09b86158d942554962b91e2e  =  569c868e20d9be8ba28fb558df8893b60aefbc2c09b86158d942554962b91e2e
